<a href="https://colab.research.google.com/github/manumeehl/trajectories/blob/main/trajectories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install tesseract-ocr
!pip install opencv-python pytesseract pytube

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,310 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 120893 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [2]:
from google.colab.patches import cv2_imshow as show
from pytube import YouTube
import pytesseract
import cv2
import re

In [3]:
def get_video(url, path):
  vid = YouTube(url)
  print(f"Downloading Video '{vid.streams[0].title}'")
  vid.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(output_path=path)
  print('Download complete')

In [4]:
def custom_crop(frame):
  height, width, _ = frame.shape
  return frame[height-140:height,0:1920]

In [7]:
def get_props(string):
  ''' Extract mission clock, altitude and velocity from a given string '''
  string = string.lower()

  # Apply a blacklist
  blacklist = ['stage 1', 'stage 2', 'falcon 9']
  for w in blacklist:
    string = string.replace(w, '')

  # Find all numbers and floats in the string
  numbers = re.findall(r'\d+(?:\.\d+)?', string)

  # Find the mission clock (T+00:00:00)
  mission_clock = re.findall(r't\+\d{2}:\d{2}:\d{2}', string)

  # Remove numbers which also occur in the mission clock
  mission_clock_numbers = re.findall(r'\d+(?:\.\d+)?', ' '.join(mission_clock))
  for num in mission_clock_numbers:
      numbers.remove(num)

  # Remove duplicates from the numbers list
  numbers = list(set(numbers))

  # Sort the numbers list in ascending order
  numbers.sort()

  # Return the sorted numbers and mission clock
  return numbers, mission_clock

In [8]:
def extract_data(frame):
  crop = custom_crop(frame)
  gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
  inverted = cv2.bitwise_not(gray)
  show(inverted)

  text = pytesseract.image_to_string(inverted)
  return get_props(text)

In [9]:
def await_liftoff():
  ''' Identify the frame at which liftoff occurs '''
  framepos = 0
  interval = 300
  clock = ''
  # Await first mission clock appearance
  while vid.isOpened():
    vid.set(cv2.CAP_PROP_FRAMES, int(framepos))
    framepos += interval

    ret, frame = vid.read()
    data = extract_data(frame)
    print(ret)

    if framepos > 1000:
      break


In [ ]:
for i in range(3):
  frame = cv2.imread(f'frames/frame_{i+1}.png')
  crop = custom_crop(frame)
  gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
  inverted = cv2.bitwise_not(gray)
  show(inverted)

  output = pytesseract.image_to_string(inverted)
  print(extract(output))

In [6]:
url = 'https://www.youtube.com/watch?v=agYuEAkEljw'
get_video(url, 'launches')

Download complete


In [11]:
vidpath = 'launches/Starlink_Mission.mp4'
vid = cv2.VideoCapture(vidpath)
fps = vid.get(cv2.CAP_PROP_FPS)
print(fps)

print('Awaiting liftoff')
await_liftoff()

0.0
Awaiting liftoff
